In [69]:
import funkcije

# Analiza zadnjih 100 oglasov na portalu avto.net (dne 26.10.2023)

Na portalu [avto.net](https://www.avto.net/) je posebna [stran](https://www.avto.net/Ads/results_100.asp?oglasrubrika=1&prodajalec=2), ki vsebuje zadnjih 100 objavljenih oglasov (stran se sproti spreminja).



In [ ]:
#s kodo v tem okvirčku izluščimo html iz spletne podstrani, ki vsebuje zadnjih 100 oglasov
import requests
from bs4 import BeautifulSoup

headers = {
    'Accept': 'text/html, image/avif, image/webp, image/apng, image/svg+xml, */ *; q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US, en;q=0.9',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Site': 'none',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
}

data_request = requests.get('https://www.avto.net/Ads/results_100.asp?', headers=headers)
soup = BeautifulSoup(data_request.content, 'html.parser')
vsebina_html = soup.prettify()
with open("zadnjih_100.html", "w", encoding='utf-8') as f:
     print(vsebina_html, file=f)


# print(data_request.status_code)

In [68]:
#s kodo v tem okvirčku bomo iz datoteke zadnjih_100.html izluščili podatke (ime, 1.registracija, prevoženih km, gorivo, menjalnik, motor, cena)


from bs4 import BeautifulSoup

with open('zadnjih_100.html', 'r', encoding='utf-8') as f:
    vsebina = f.read()
    soup = BeautifulSoup(vsebina, 'html.parser')

#Pridelamo seznam z imeni vseh avtov (kot so ta zapisana na avto.net)

imena_avtov = []

#Ker imajo vsa imena avtov pred njimi značko <div class="GO-Results-Naziv ... to izkoristimo, da izluščimo imena ven
for element in soup.find_all(class_='GO-Results-Naziv'):
    imena_avtov.append(element.text.strip())


#Sedaj pa pridelamo še en seznam, z vsemi drugimi podatki (hočem, da mi pri motorju pobere samo konjske moči)

drugi_podatki = []
for element in soup.find_all('td', class_='pl-3'):
    drugi_podatki.append(element.text.strip())

#drugi_podatki je seznam tipa ['1.registracija', '2011', 'Prevoženih', '270000 km', 'Gorivo', 'bencinski motor']...

##hočem, da mi pri moči motorja pobere samo konjske moči,
drugi_podatki = poberi_KM(drugi_podatki)

#morebiten podatek samo v kW mi pretvori v KM (1 kW = 1.341 KM) in zaokroži na najbližje celo število
drugi_podatki = pretvorjeno_v_KM(drugi_podatki)

cene_avtov = []

for element in soup.find_all(class_=['GO-Results-Price', 'GO-Results-Top-Price']):
    cene_avtov.append(element.text.strip())

#V kodi se cena za posamezen avto 2x ponovi (razen pri top ponudbi), zato uporabim funkcijo uredi_seznam, ki mi za vsak avto pusti samo eno ceno
cene_avtov = uredi_seznam(cene_avtov)

print(cene_avtov)
cene_avtov = cene_avtov[::2] #vzamemo vsak drugi element, ker se vsaka cena dvakrat ponovi
#top ponudba manjka cena
# for i in range(len(cene_avtov)):
#     print('AKCIJSKA CENA\n' in cene_avtov[i])
#     if 'AKCIJSKA CENA\n' in cene_avtov[i]: #'AKCIJSKA CENA\n          \n\n\n\n\n\n            13.990 €\n           \n\n            13.690 €' --> hočemo samo ceno na koncu
#         cene_avtov[i] = cene_avtov[i][(-10):]
#         while cene_avtov[i][0] == ' ':
#             cene_avtov[i] = cene_avtov[1:]
# print(cene_avtov)



['7.990 €', '7.990 €', '2.600 €', '2.600 €', '7.980 €', '7.980 €', '1.300 €', '1.300 €', '17.490 €', '17.490 €', '16.800 €', '16.800 €', '26.300 €', '26.300 €', '34.900 €', '34.900 €', '8.690 €', '8.690 €', '9.900 €', '9.900 €', '12.500 €', '12.500 €', '21.999 €', '21.999 €', '2.450 €', '2.450 €', '550 €', '550 €', '9.799 €', '9.799 €', '5.850 €', '5.850 €', '35.000 €', '35.000 €', '12.999 €', '12.999 €', '8.699 €', '4.500 €', '4.500 €', '580 €', '580 €', 'AKCIJSKA CENA\n          \n\n\n\n\n\n            13.990 €\n           \n\n            13.690 €', 'AKCIJSKA CENA\n          \n\n\n\n\n\n            13.990 €\n           \n\n            13.690 €', '5.000 €', '5.000 €', '8.940 €', '8.940 €', '9.450 €', '9.450 €', '3.390 €\n           \n\n\n\n\n\n\n\n\n\n            oz. 51,00 EUR / mesec (*)', '3.390 €\n           \n\n\n\n\n\n\n\n\n\n            oz. 51,00 EUR / mesec (*)', '13.945 €', '13.945 €', '3.000 €', '3.000 €', '4.000 €', '4.000 €', '1.670 €\n           \n\n\n\n\n\n\n\n\n\n       

In [ ]:
#Zapišemo slovar oblike {'ime avta':{'1. registracija': 2011, 'Prevoženih': '270000 km', 'Gorivo': 'bencinski motor', 'Menjalnik': 'avtomatski menjalnik', 'Moč motorja': '160 KM'}}
#slovar sem izbral zato, ker so se določeni podatki v kodi dvakrat ponovili. Nekateri avti pa tudi nimajo podatka o npr. prevoženih kilometrih. Pri nekaterih električnih avtih
#pa pride podatek tipa 'Baterija': '52 kw', ki ga v slovar nisem zapisal, moč motorja pa sem iz kW pretvoril v KM (1 kW = 1.341 KM) in zaokrožil na najbližje celo število.
#električni avti imajo vedno avtomatski menjalnik, zato bom v slovar dodal tudi to (tudi če tega na avto.net eksplicitno ne piše)


#za vsako ime avta bo treba zgenerirat svoj vmesni slovar

#koda za izdelavo slovarja
sez_indeksov = na_katerih_indeksih_se_nahaja(drugi_podatki, '1.registracija')
slovar = {}
vmesni_slovar = {}
seznam_kljucev = ['1.registracija', 'Prevoženih', 'Gorivo', 'Menjalnik', 'Motor']
i = 1
j = 0
for el in drugi_podatki:
    if i in sez_indeksov: #natanko tedaj, ko naletimo na prvo registracijo, hočemo začet nov vmesni slovar
        slovar[imena_avtov[j]] = vmesni_slovar
        vmesni_slovar = {}
        j += 1
    if drugi_podatki[i - 1] in seznam_kljucev:
        vmesni_slovar[drugi_podatki[i - 1]] = drugi_podatki[i]
    i += 1

#dodamo avtomatski menjalnik k avtom z električnim pogonom
for vmesni_slovar in slovar.values():
    if 'elektro pogon' in vmesni_slovar.values():
        vmesni_slovar['Menjalnik'] = 'avtomatski menjalnik'
        
print(slovar)

In [59]:
#Tu bomo iz slovarja napisali csv datoteko, ki jo bomo kasneje uporabili za analizo podatkov

import csv
with open('zadnjih_100.csv', 'w', newline='', encoding='utf-8') as g:
    zapis = csv.writer(g)
    
    #Napišemo prvo vrstico
    zapis.writerow(['Ime', '1.registracija', 'Prevoženih', 'Gorivo', 'Menjalnik', 'Motor'])
    
    #Zapišemo podatkev vse naslednje vrstice
    for ime, podatki in slovar.items():
        zapis.writerow([ime] + [podatki.get(kljuc, '') for kljuc in ['1.registracija', 'Prevoženih', 'Gorivo', 'Menjalnik', 'Motor']])

